In [ ]:
import sounddevice as sd
import numpy as np

In [ ]:
# === PARAMETERS ===
sample_rate = 44100
delay_time_sec = 0.5
feedback = 0.5
mix = 0.75
blocksize = 1024

# === DELAY BUFFER ===
max_delay_samples = int(sample_rate * 2)
delay_buffer = np.zeros(max_delay_samples)
write_idx = 0

def delay_effect(input_block):
    global write_idx
    output = np.zeros_like(input_block)

    for i in range(len(input_block)):
        delay_idx = (write_idx - int(delay_time_sec * sample_rate)) % max_delay_samples
        delayed_sample = delay_buffer[delay_idx]

        delay_buffer[write_idx] = input_block[i] + delayed_sample * feedback
        output[i] = (1 - mix) * input_block[i] + mix * delayed_sample

        write_idx = (write_idx + 1) % max_delay_samples

    return output
    
def highpass(signal, alpha=0.995):
    # Simple first-order high-pass filter
    y = np.zeros_like(signal)
    prev_input = 0
    prev_output = 0
    for i in range(len(signal)):
        y[i] = alpha * (prev_output + signal[i] - prev_input)
        prev_input = signal[i]
        prev_output = y[i]
    return y

    
# === AUDIO CALLBACK ===
def callback(indata, outdata, frames, time, status):
    if status:
        print(status)

    mono_input = indata[:, 0]  # Take input from channel 0 (SSL-2 Input 1)
    
    processed = delay_effect(mono_input)

    # Clean up any noise with high-pass filter and clipping excess values
    processed = highpass(processed)
    processed = np.clip(processed, -1.0, 1.0)

    stereo_output = np.column_stack([processed, processed])  # Output stereo
    outdata[:] = stereo_output

# === RUN STREAM (Mono In, Stereo Out) ===
stream = sd.Stream(samplerate=sample_rate, blocksize=blocksize,
                   channels=(1, 2), dtype='float32',
                   callback=callback, latency='low')

print("Running live delay... Press Enter to stop.")
with stream:
    input()